In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import sys

### Functions required to parse the 'cfg.log' file from the experiments

In [3]:
def parse_config_log_with_final(filename):
    result = {}
    train_loss = []
    train_acc = []
    test_acc = []
    test_loss = []
    train_round_time = []
    test_round_time = []
    successfully_upload_clients_count = []
    loss_threshold = []
    loss_threshold_dict = {}
    loss_low = []
    loss_low_dict = {}
    loss_high = []
    loss_high_dict = {}
    deadlines = []
    deadlines_dict = {}
    deadlines_high = []
    deadlines_high_dict = {}
    deadlines_low = []
    deadlines_low_dict = {}
    loss_threshold_percentage = []
    loss_threshold_percentage_dict = {}
    deadline_percentage = []
    deadline_percentage_dict = {}
    current_round_loss = []
    
    f = open(filename)

    f_lines = f.readlines()
    
    current_round = 0
    
    is_final_appeared = False
    
    for line in f_lines:
        tmp = line.split(' ')
        if '=====================' in tmp and 'Round' in tmp and 'of' in tmp:
            current_round += 1
        if 'average' in tmp and 'acc:' in tmp:
            train_acc.append(float(tmp[tmp.index('average')+2][:-1]))
            train_loss.append(float(tmp[tmp.index('average')+5][:-1]))
        elif 'test_accuracy:' in tmp:
            test_acc.append(float(tmp[tmp.index('test_accuracy:')+1][:-1]))
        elif 'test_loss:' in tmp:
            test_loss.append(float(tmp[tmp.index('test_loss:')+1][:-1]))
        elif 'current' in tmp and 'time:' in tmp:
            train_round_time.append(float(tmp[tmp.index('time:')+1].split('\t')[0]))
            if len(train_round_time)%5 == 0:
                test_round_time.append(float(tmp[tmp.index('time:')+1].split('\t')[0]))
        elif 'upload' in tmp and 'successfully\n' in tmp:
            if tmp.index('successfully\n') - tmp.index('upload') == 1:
                successfully_upload_clients_count.append(float(tmp[tmp.index('upload')-4]))
            else:
                continue
        elif 'round' in tmp and 'failed,' in tmp and 'global' in tmp:
            train_acc.append(train_acc[-1])
            train_loss.append(train_loss[-1])
        elif 'loss_threshold' in tmp and 'loss_high' in tmp and 'loss_low' in tmp:
            loss_threshold.append(float(tmp[tmp.index('loss_threshold')+1][:-1]))
            loss_threshold_dict[current_round] = (float(tmp[tmp.index('loss_threshold')+1][:-1]))
            loss_low.append(float(tmp[tmp.index('loss_low')+1][:-1]))
            loss_low_dict[current_round] = (float(tmp[tmp.index('loss_low')+1][:-1]))
            loss_high.append(float(tmp[tmp.index('loss_high')+1][:-1]))
            loss_high_dict[current_round] = (float(tmp[tmp.index('loss_high')+1][:-1]))
        elif 'deadline' in tmp and 'percentage' in tmp and 'loss_threshold' in tmp:
            deadline_percentage.append(float(tmp[tmp.index('deadline')+2][:-1]))
            deadline_percentage_dict[current_round] = (float(tmp[tmp.index('deadline')+2][:-1]))
            loss_threshold_percentage.append(float(tmp[tmp.index('loss_threshold')+2].strip()))
            loss_threshold_percentage_dict[current_round] = (float(tmp[tmp.index('loss_threshold')+2].strip()))
        elif 'this' in tmp and 'round' in tmp and 'deadline' in tmp and'percentage' not in tmp:
            deadlines.append(float(tmp[tmp.index('deadline')+1][:-1]))
            deadlines_dict[current_round] = (float(tmp[tmp.index('deadline')+1][:-1]))
        elif 'deadline_low' in tmp and 'deadline_high' in tmp:
            deadlines_low.append(float(tmp[tmp.index('deadline_low')+1][:-1]))
            deadlines_low_dict[current_round] = (float(tmp[tmp.index('deadline_low')+1][:-1]))
            deadlines_high.append(float(tmp[tmp.index('deadline_high')+1][:-1]))
            deadlines_high_dict[current_round] = (float(tmp[tmp.index('deadline_high')+1][:-1]))
        elif 'loss_threshold_percentage' in tmp:
            loss_threshold_percentage.append(float(tmp[tmp.index('loss_threshold_percentage')+1].strip()))
        elif 'current_round_loss:' in tmp:
            current_round_loss.append(float(tmp[tmp.index('current_round_loss:')+1].strip()))
        elif 'FINAL' in tmp:
            is_final_appeared = True
    
    if is_final_appeared and len(test_round_time) != len(test_acc):
        test_round_time.append(train_round_time[-1])
    
    result['train_loss'] = train_loss
    result['train_acc'] = train_acc
    result['test_loss'] = test_loss
    result['test_acc'] = test_acc
    result['train_round_time'] = train_round_time
    result['test_round_time'] = test_round_time
    result['successfully_upload_clients_count'] = successfully_upload_clients_count
    result['loss_threshold'] = loss_threshold
    result['loss_threshold_dict'] = loss_threshold_dict
    result['loss_low'] = loss_low
    result['loss_low_dict'] = loss_low_dict
    result['loss_high'] = loss_high
    result['loss_high_dict'] = loss_high_dict
    result['deadlines_low'] = deadlines_low
    result['deadlines_low_dict'] = deadlines_low_dict
    result['deadlines_high'] = deadlines_high
    result['deadlines_high_dict'] = deadlines_high_dict
    result['loss_threshold_percentage'] = loss_threshold_percentage
    result['loss_threshold_percentage_dict'] = loss_threshold_percentage_dict
    result['deadlines'] = deadlines
    result['deadlines_dict'] = deadlines_dict
    result['deadline_percentage'] = deadline_percentage
    result['deadline_percentage_dict'] = deadline_percentage_dict
    result['current_round_loss'] = current_round_loss
    
    return result

In [4]:
def measure_time(log_dict, objective, what_to_check):
    i = -1
    for i, acc in enumerate(log_dict[what_to_check]):
        if acc >= objective:
            return log_dict['test_round_time'][i]
    return sys.maxsize

In [5]:
def measure_final_acc(log_dict, what_to_check, global_final_time):
    for i in range(len(log_dict['test_round_time'])):
        if log_dict['test_round_time'][i] >= global_final_time:
            return log_dict[what_to_check][i-1]
    return log_dict[what_to_check][-1]

### Experimental Result

In [7]:
fedavg_ddl_fixed_1_0 = [
    parse_config_log_with_final('../models/configs/har/har_fedavg_1T_seed0.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_fedavg_1T_seed1.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_fedavg_1T_seed2.cfg.log'),
]

fedavg_ddl_fixed_2_0 = [
    parse_config_log_with_final('../models/configs/har/har_fedavg_2T_seed0.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_fedavg_2T_seed1.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_fedavg_2T_seed2.cfg.log'),
]

fedavg_ddl_smartpc = [
    parse_config_log_with_final('../models/configs/har/har_fedavg_SPC_seed0.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_fedavg_SPC_seed1.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_fedavg_SPC_seed2.cfg.log'),
]

fedavg_ddl_waitforall = [
    parse_config_log_with_final('../models/configs/har/har_fedavg_WFA_seed0.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_fedavg_WFA_seed1.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_fedavg_WFA_seed2.cfg.log'),
]

fedprox_ddl_fixed_1_0 = [
    parse_config_log_with_final('../models/configs/har/har_fedprox_mu_0_0_1T_seed0.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_fedprox_mu_0_0_1T_seed1.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_fedprox_mu_0_0_1T_seed2.cfg.log'),
]

fedprox_ddl_fixed_2_0 = [
    parse_config_log_with_final('../models/configs/har/har_fedprox_mu_0_0_2T_seed0.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_fedprox_mu_0_0_2T_seed1.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_fedprox_mu_0_0_2T_seed2.cfg.log'),
]

sampleselection = [
    parse_config_log_with_final('../models/configs/har/har_sampleselection_baseline_seed0.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_sampleselection_baseline_seed1.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_sampleselection_baseline_seed2.cfg.log')
]

fb_f = [
    parse_config_log_with_final('../models/configs/har/har_fedbalancer_seed0.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_fedbalancer_seed1.cfg.log'),
    parse_config_log_with_final('../models/configs/har/har_fedbalancer_seed2.cfg.log'),
]

global_final_times = [
    10187, 12398, 11859
]

In [9]:
speedup_result = np.zeros((3, 8))
accuracy_result = np.zeros((3, 8))

# Test with three random seeds
i = 0
what_to_check = 'test_acc'
objective = measure_final_acc(fedavg_ddl_fixed_1_0[i], what_to_check, global_final_times[i])
global_final_time = global_final_times[i]

tta1 = measure_time(fedavg_ddl_fixed_1_0[i], objective, what_to_check)
tta2 = measure_time(fedavg_ddl_fixed_2_0[i], objective, what_to_check)
tta3 = measure_time(fedavg_ddl_smartpc[i], objective, what_to_check)
tta4 = measure_time(fedavg_ddl_waitforall[i], objective, what_to_check)
tta5 = measure_time(fedprox_ddl_fixed_1_0[i], objective, what_to_check)
tta6 = measure_time(fedprox_ddl_fixed_2_0[i], objective, what_to_check)
tta7 = measure_time(sampleselection[i], objective, what_to_check)
tta8 = measure_time(fb_f[i], objective, what_to_check)

acc1 = measure_final_acc(fedavg_ddl_fixed_1_0[i], what_to_check, global_final_time)
acc2 = measure_final_acc(fedavg_ddl_fixed_2_0[i], what_to_check, global_final_time)
acc3 = measure_final_acc(fedavg_ddl_smartpc[i], what_to_check, global_final_time)
acc4 = measure_final_acc(fedavg_ddl_waitforall[i], what_to_check, global_final_time)
acc5 = measure_final_acc(fedprox_ddl_fixed_1_0[i], what_to_check, global_final_time)
acc6 = measure_final_acc(fedprox_ddl_fixed_2_0[i], what_to_check, global_final_time)
acc7 = measure_final_acc(sampleselection[i], what_to_check, global_final_time)
acc8 = measure_final_acc(fb_f[i], what_to_check, global_final_time)

print("Random Seed: ", i)
print("{:15s} | {:10s} | {:7s}".format('METHOD', 'SPEEDUP', 'ACCURACY'))
print("{:15s}".format('FedAvg+1T')+" | %.2f\t | %.3f\t" % (round(tta1 / tta1, 3), round(acc1, 3)))
print("{:15s}".format('FedAvg+2T')+" | %.2f\t | %.3f\t" % (round(tta1 / tta2, 3), round(acc2, 3)))
print("{:15s}".format('FedAvg+SPC')+" | %.2f\t | %.3f\t" % (round(tta1 / tta3, 3), round(acc3, 3)))
print("{:15s}".format('FedAvg+WFA')+" | %.2f\t | %.3f\t" % (round(tta1 / tta4, 3), round(acc4, 3)))
print("{:15s}".format('Prox+1T')+" | %.2f\t | %.3f\t" % (round(tta1 / tta5, 3), round(acc5, 3)))
print("{:15s}".format('Prox+2T')+" | %.2f\t | %.3f\t" % (round(tta1 / tta6, 3), round(acc6, 3)))
print("{:15s}".format('SampleSelection')+" | %.2f\t | %.3f\t" % (round(tta1 / tta7, 3), round(acc7, 3)))
print("{:15s}".format('FedBalancer')+" | %.2f\t | %.3f\t" % (round(tta1 / tta8, 3), round(acc8, 3)))
print()

speedup_result[i,0] = tta1 / tta1
speedup_result[i,1] = tta1 / tta2
speedup_result[i,2] = tta1 / tta3
speedup_result[i,3] = tta1 / tta4
speedup_result[i,4] = tta1 / tta5
speedup_result[i,5] = tta1 / tta6
speedup_result[i,6] = tta1 / tta7
speedup_result[i,7] = tta1 / tta8

accuracy_result[i,0] = acc1
accuracy_result[i,1] = acc2
accuracy_result[i,2] = acc3
accuracy_result[i,3] = acc4
accuracy_result[i,4] = acc5
accuracy_result[i,5] = acc6
accuracy_result[i,6] = acc7
accuracy_result[i,7] = acc8

i = 1
what_to_check = 'test_acc'
objective = measure_final_acc(fedavg_ddl_smartpc[i], what_to_check, global_final_times[i])
global_final_time = global_final_times[i]

tta1 = measure_time(fedavg_ddl_fixed_1_0[i], objective, what_to_check)
tta2 = measure_time(fedavg_ddl_fixed_2_0[i], objective, what_to_check)
tta3 = measure_time(fedavg_ddl_smartpc[i], objective, what_to_check)
tta4 = measure_time(fedavg_ddl_waitforall[i], objective, what_to_check)
tta5 = measure_time(fedprox_ddl_fixed_1_0[i], objective, what_to_check)
tta6 = measure_time(fedprox_ddl_fixed_2_0[i], objective, what_to_check)
tta7 = measure_time(sampleselection[i], objective, what_to_check)
tta8 = measure_time(fb_f[i], objective, what_to_check)

acc1 = measure_final_acc(fedavg_ddl_fixed_1_0[i], what_to_check, global_final_time)
acc2 = measure_final_acc(fedavg_ddl_fixed_2_0[i], what_to_check, global_final_time)
acc3 = measure_final_acc(fedavg_ddl_smartpc[i], what_to_check, global_final_time)
acc4 = measure_final_acc(fedavg_ddl_waitforall[i], what_to_check, global_final_time)
acc5 = measure_final_acc(fedprox_ddl_fixed_1_0[i], what_to_check, global_final_time)
acc6 = measure_final_acc(fedprox_ddl_fixed_2_0[i], what_to_check, global_final_time)
acc7 = measure_final_acc(sampleselection[i], what_to_check, global_final_time)
acc8 = measure_final_acc(fb_f[i], what_to_check, global_final_time)

print("Random Seed: ", i)
print("{:15s} | {:10s} | {:7s}".format('METHOD', 'SPEEDUP', 'ACCURACY'))
print("{:15s}".format('FedAvg+1T')+" | %.2f\t | %.3f\t" % (round(tta3 / tta1, 3), round(acc1, 3)))
print("{:15s}".format('FedAvg+2T')+" | %.2f\t | %.3f\t" % (round(tta3 / tta2, 3), round(acc2, 3)))
print("{:15s}".format('FedAvg+SPC')+" | %.2f\t | %.3f\t" % (round(tta3 / tta3, 3), round(acc3, 3)))
print("{:15s}".format('FedAvg+WFA')+" | %.2f\t | %.3f\t" % (round(tta3 / tta4, 3), round(acc4, 3)))
print("{:15s}".format('Prox+1T')+" | %.2f\t | %.3f\t" % (round(tta3 / tta5, 3), round(acc5, 3)))
print("{:15s}".format('Prox+2T')+" | %.2f\t | %.3f\t" % (round(tta3 / tta6, 3), round(acc6, 3)))
print("{:15s}".format('SampleSelection')+" | %.2f\t | %.3f\t" % (round(tta3 / tta7, 3), round(acc7, 3)))
print("{:15s}".format('FedBalancer')+" | %.2f\t | %.3f\t" % (round(tta3 / tta8, 3), round(acc8, 3)))
print()

speedup_result[i,0] = tta3 / tta1
speedup_result[i,1] = tta3 / tta2
speedup_result[i,2] = tta3 / tta3
speedup_result[i,3] = tta3 / tta4
speedup_result[i,4] = tta3 / tta5
speedup_result[i,5] = tta3 / tta6
speedup_result[i,6] = tta3 / tta7
speedup_result[i,7] = tta3 / tta8

accuracy_result[i,0] = acc1
accuracy_result[i,1] = acc2
accuracy_result[i,2] = acc3
accuracy_result[i,3] = acc4
accuracy_result[i,4] = acc5
accuracy_result[i,5] = acc6
accuracy_result[i,6] = acc7
accuracy_result[i,7] = acc8

i = 2
what_to_check = 'test_acc'
objective = measure_final_acc(fedavg_ddl_smartpc[i], what_to_check, global_final_times[i])
global_final_time = global_final_times[i]

tta1 = measure_time(fedavg_ddl_fixed_1_0[i], objective, what_to_check)
tta2 = measure_time(fedavg_ddl_fixed_2_0[i], objective, what_to_check)
tta3 = measure_time(fedavg_ddl_smartpc[i], objective, what_to_check)
tta4 = measure_time(fedavg_ddl_waitforall[i], objective, what_to_check)
tta5 = measure_time(fedprox_ddl_fixed_1_0[i], objective, what_to_check)
tta6 = measure_time(fedprox_ddl_fixed_2_0[i], objective, what_to_check)
tta7 = measure_time(sampleselection[i], objective, what_to_check)
tta8 = measure_time(fb_f[i], objective, what_to_check)

acc1 = measure_final_acc(fedavg_ddl_fixed_1_0[i], what_to_check, global_final_time)
acc2 = measure_final_acc(fedavg_ddl_fixed_2_0[i], what_to_check, global_final_time)
acc3 = measure_final_acc(fedavg_ddl_smartpc[i], what_to_check, global_final_time)
acc4 = measure_final_acc(fedavg_ddl_waitforall[i], what_to_check, global_final_time)
acc5 = measure_final_acc(fedprox_ddl_fixed_1_0[i], what_to_check, global_final_time)
acc6 = measure_final_acc(fedprox_ddl_fixed_2_0[i], what_to_check, global_final_time)
acc7 = measure_final_acc(sampleselection[i], what_to_check, global_final_time)
acc8 = measure_final_acc(fb_f[i], what_to_check, global_final_time)

print("Random Seed: ", i)
print("{:15s} | {:10s} | {:7s}".format('METHOD', 'SPEEDUP', 'ACCURACY'))
print("{:15s}".format('FedAvg+1T')+" | %.2f\t | %.3f\t" % (round(tta3 / tta1, 3), round(acc1, 3)))
print("{:15s}".format('FedAvg+2T')+" | %.2f\t | %.3f\t" % (round(tta3 / tta2, 3), round(acc2, 3)))
print("{:15s}".format('FedAvg+SPC')+" | %.2f\t | %.3f\t" % (round(tta3 / tta3, 3), round(acc3, 3)))
print("{:15s}".format('FedAvg+WFA')+" | %.2f\t | %.3f\t" % (round(tta3 / tta4, 3), round(acc4, 3)))
print("{:15s}".format('Prox+1T')+" | %.2f\t | %.3f\t" % (round(tta3 / tta5, 3), round(acc5, 3)))
print("{:15s}".format('Prox+2T')+" | %.2f\t | %.3f\t" % (round(tta3 / tta6, 3), round(acc6, 3)))
print("{:15s}".format('SampleSelection')+" | %.2f\t | %.3f\t" % (round(tta3 / tta7, 3), round(acc7, 3)))
print("{:15s}".format('FedBalancer')+" | %.2f\t | %.3f\t" % (round(tta3 / tta8, 3), round(acc8, 3)))
print()

speedup_result[i,0] = tta3 / tta1
speedup_result[i,1] = tta3 / tta2
speedup_result[i,2] = tta3 / tta3
speedup_result[i,3] = tta3 / tta4
speedup_result[i,4] = tta3 / tta5
speedup_result[i,5] = tta3 / tta6
speedup_result[i,6] = tta3 / tta7
speedup_result[i,7] = tta3 / tta8

accuracy_result[i,0] = acc1
accuracy_result[i,1] = acc2
accuracy_result[i,2] = acc3
accuracy_result[i,3] = acc4
accuracy_result[i,4] = acc5
accuracy_result[i,5] = acc6
accuracy_result[i,6] = acc7
accuracy_result[i,7] = acc8

print("Averaged Results")
print("{:15s} | {:11s} | {:10s}".format('METHOD', 'SPEEDUP', 'ACCURACY'))
print("{:15s}".format('FedAvg+1T')+" | %.2f ± %.2f | %.3f ± %.3f" % (round(np.mean(speedup_result[:,0]), 3),round(np.std(speedup_result[:,0]), 3),round(np.mean(accuracy_result[:,0]), 3),round(np.std(accuracy_result[:,0]), 3)))
print("{:15s}".format('FedAvg+2T')+" | %.2f ± %.2f | %.3f ± %.3f" % (round(np.mean(speedup_result[:,1]), 3),round(np.std(speedup_result[:,1]), 3),round(np.mean(accuracy_result[:,1]), 3),round(np.std(accuracy_result[:,1]), 3)))
print("{:15s}".format('FedAvg+SPC')+" | %.2f ± %.2f | %.3f ± %.3f" % (round(np.mean(speedup_result[:,2]), 3),round(np.std(speedup_result[:,2]), 3),round(np.mean(accuracy_result[:,2]), 3),round(np.std(accuracy_result[:,2]), 3)))
print("{:15s}".format('FedAvg+WFA')+" | %.2f ± %.2f | %.3f ± %.3f" % (round(np.mean(speedup_result[:,3]), 3),round(np.std(speedup_result[:,3]), 3),round(np.mean(accuracy_result[:,3]), 3),round(np.std(accuracy_result[:,3]), 3)))
print("{:15s}".format('Prox+1T')+" | %.2f ± %.2f | %.3f ± %.3f" % (round(np.mean(speedup_result[:,4]), 3),round(np.std(speedup_result[:,4]), 3),round(np.mean(accuracy_result[:,4]), 3),round(np.std(accuracy_result[:,4]), 3)))
print("{:15s}".format('Prox+2T')+" | %.2f ± %.2f | %.3f ± %.3f" % (round(np.mean(speedup_result[:,5]), 3),round(np.std(speedup_result[:,5]), 3),round(np.mean(accuracy_result[:,5]), 3),round(np.std(accuracy_result[:,5]), 3)))
print("{:15s}".format('SampleSelection')+" | %.2f ± %.2f | %.3f ± %.3f" % (round(np.mean(speedup_result[:,6]), 3),round(np.std(speedup_result[:,6]), 3),round(np.mean(accuracy_result[:,6]), 3),round(np.std(accuracy_result[:,6]), 3)))
print("{:15s}".format('FedBalancer')+" | %.2f ± %.2f | %.3f ± %.3f" % (round(np.mean(speedup_result[:,7]), 3),round(np.std(speedup_result[:,7]), 3),round(np.mean(accuracy_result[:,7]), 3),round(np.std(accuracy_result[:,7]), 3)))

Random Seed:  0
METHOD          | SPEEDUP    | ACCURACY
FedAvg+1T       | 1.00	 | 0.843	
FedAvg+2T       | 0.67	 | 0.821	
FedAvg+SPC      | 0.73	 | 0.818	
FedAvg+WFA      | 0.66	 | 0.820	
Prox+1T         | 1.08	 | 0.844	
Prox+2T         | 0.67	 | 0.820	
SampleSelection | 0.87	 | 0.838	
FedBalancer     | 1.89	 | 0.882	

Random Seed:  1
METHOD          | SPEEDUP    | ACCURACY
FedAvg+1T       | 0.17	 | 0.773	
FedAvg+2T       | 0.78	 | 0.835	
FedAvg+SPC      | 1.00	 | 0.845	
FedAvg+WFA      | 0.77	 | 0.835	
Prox+1T         | 0.96	 | 0.860	
Prox+2T         | 0.78	 | 0.835	
SampleSelection | 0.96	 | 0.856	
FedBalancer     | 1.20	 | 0.876	

Random Seed:  2
METHOD          | SPEEDUP    | ACCURACY
FedAvg+1T       | 0.30	 | 0.825	
FedAvg+2T       | 0.60	 | 0.801	
FedAvg+SPC      | 1.00	 | 0.856	
FedAvg+WFA      | 0.59	 | 0.802	
Prox+1T         | 0.86	 | 0.842	
Prox+2T         | 0.60	 | 0.802	
SampleSelection | 0.86	 | 0.841	
FedBalancer     | 1.58	 | 0.807	

Averaged Results
METHOD          | SP